<a href="https://colab.research.google.com/github/nipun214/Python-codes-for-ML/blob/master/textrank_algorithm_for_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [57]:
from google.colab import files
uploaded = files.upload()

Saving tennis_articles_1.csv to tennis_articles_1.csv


In [58]:
df = pd.read_csv("tennis_articles_1.csv",encoding='latin1')

In [59]:
df.head()


,article_id,article_title,article_text,source
0,1,"I do not have friends in tennis, says Maria Sh...",Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,Federer defeats Medvedev to advance to 14th Sw...,"BASEL, Switzerland (AP) ? Roger Federer advanc...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Tennis: Roger Federer ignored deadline set by ...,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Nishikori to face off against Anderson in Vien...,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,Roger Federer has made this huge change to ten...,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...


In [63]:
df['article_text'][14]

'Why hasn\x92t Petra Kvitova done better at the US Open during her career?\n\n\x93It\x92s always pretty crowded,\x94 the Czech told ESPN\x92s Rennae Stubbs after her mostly impressive 6-4, 6-3 win over Jessica Pegula on Friday night. \x93I like the peace and quiet.\x94\n\nKvitova isn\x92t shy, exactly, but she is easygoing and low-key. Most years, those are not words that anyone would use to describe the US Open. But 2020 is obviously not most years, and the grounds are definitely not crowded. If Kvitova wants peace and quiet, she can find a lot of it in the bubble.\n\nIs this, by itself, a reason to believe that Kvitova can make it past the quarterfinals in New York for the first time, and win the whole tournament? Probably not. But at 30, with six of the Top 10 absent, this would seem to be the year for her to make a push in New York. So far we\x92ve talked a lot about Serena Williams, Naomi Osaka, and Victoria Azarenka as potential contenders for the women\x92s crown. Should we also

In [23]:
df['article_text'][2]

'Roger Federer has revealed that organisers of the re-launched and condensed Davis Cup gave him three days to decide if he would commit to the controversial competition. Speaking at the Swiss Indoors tournament where he will play in Sunday\x92s final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment. \x93They only left me three days to decide,\x94 Federer said. \x93I didn\x92t to have time to consult with all the people I had to consult. \x93I could not make a decision in that time, so I told them to do what they wanted.\x94 The 20-time Grand Slam champion has voiced doubts about the wisdom of the one-week format to be introduced by organisers Kosmos, who have promised the International Tennis Federation up to $3 billion in prize money over the next quarter-century. The competition is set to feature 18 countries in the November 18-24 finals in Madrid next year, and will repl

In [64]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in df['article_text']:
  sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x] # flatten list

In [66]:
sentences[:14]

['Maria Sharapova has basically no friends as tennis players on the WTA Tour.',
 "The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much.",
 'I think everyone knows this is my job here.',
 "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.",
 "So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.",
 "I'm a pretty competitive girl.",
 "I say my hellos, but I'm not sending any players flowers as well.",
 "Uhm, I'm not really friendly or close to many players.",
 "I have not a lot of friends away from the courts.'",
 'When she said she is not really close to a lot of players, is that something strategic that she is doing?',
 "Is it different on the men's tour than the women's tour?",
 "'No, not at

In [67]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

In [69]:
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [70]:
len(word_embeddings)

400000

In [71]:
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [72]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [73]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [74]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [75]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [76]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [77]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [78]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [79]:
from sklearn.metrics.pairwise import cosine_similarity

In [80]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [81]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [82]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [101]:
# Extract top 10 sentences as the summary
for i in range(10):
  print(i, ranked_sentences[i][1])

0 Is this, by itself, a reason to believe that Kvitova can make it past the quarterfinals in New York for the first time, and win the whole tournament?
1 As every tennis fan of a certain age knows, the sport has already had a Big Cat: Miloslav Mecir, the 63 Slovak who eased the ball around the court deceptively enough to reach two Grand Slam finals and win gold at the 1988 Seoul Olympics, before back problems forced him to retire at 26.
2 On a night like Thursday, he would have been hard for anyone on his Djokovic-less side of the draw to beat.
3 Exhausted after spending half his round deep in the bushes searching for my ball, as well as those of two other golfers he?d never met before, our incredibly giving designated driver asked if we didn?t mind going straight home after signing off so he could rest up a little before heading to work.
4 While Nadal will be making his post-lockdown debut, Djokovic will be trying to put the one he made in New York behind him.
5 In the run-up to the